In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="mesolitica/malay-orpheus-3b-0.1-ft-lora-128", 
                  allow_patterns="*.safetensors",
                  local_dir = './')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

'/home/husein/ssd3/tts'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

def redistribute_codes(row):
    row_length = row.size(0)
    new_length = (row_length // 7) * 7
    trimmed_row = row[:new_length]
    code_list = [t - 128266 for t in trimmed_row]
    layer_1 = []
    layer_2 = []
    layer_3 = []
    for i in range((len(code_list)+1)//7):
        layer_1.append(code_list[7*i][None])
        layer_2.append(code_list[7*i+1][None]-4096)
        layer_3.append(code_list[7*i+2][None]-(2*4096))
        layer_3.append(code_list[7*i+3][None]-(3*4096))
        layer_2.append(code_list[7*i+4][None]-(4*4096))
        layer_3.append(code_list[7*i+5][None]-(5*4096))
        layer_3.append(code_list[7*i+6][None]-(6*4096))
    
    with torch.no_grad():

        codes = [torch.concat(layer_1)[None], 
            torch.concat(layer_2)[None], 
            torch.concat(layer_3)[None]]
        audio_hat = snac_model.decode(codes)
        return audio_hat.cpu()[0, 0]

In [4]:
tokenizer = AutoTokenizer.from_pretrained('canopylabs/orpheus-3b-0.1-ft')

In [5]:
ori_model = AutoModelForCausalLM.from_pretrained(
    'canopylabs/orpheus-3b-0.1-ft', tie_word_embeddings=False
).cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
ori_model.lm_head.weight.data = ori_model.model.embed_tokens.weight.data.clone()
state_dict = ori_model.state_dict()

In [7]:
from safetensors import safe_open
from tqdm import tqdm

f = safe_open(f"adapter_model.safetensors", framework="pt", device=0)
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A)
    B = f.get_tensor(B)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 1.5)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 458.36it/s]


In [8]:
from snac import SNAC
snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
snac_model = snac_model.to("cuda")

In [25]:
speaker = 'Husein'
text = 'Nama saya Husein, <chuckle>, erm, saya tak suka nasi ayam dan tak suka mandi.'
prompt = f'<custom_token_3><|begin_of_text|>{speaker}: {text}<|eot_id|><custom_token_4><custom_token_5><custom_token_1>'
input_ids = tokenizer(prompt,add_special_tokens = False, return_tensors = 'pt').to('cuda')

In [ ]:
with torch.no_grad():
    generated_ids = ori_model.generate(
      **input_ids,
      max_new_tokens=1200,
      do_sample=True,
      temperature=0.9,
      top_p=0.95,
      repetition_penalty=1.1,
      num_return_sequences=1,
      eos_token_id=128258,
    )

Setting `pad_token_id` to `eos_token_id`:128258 for open-end generation.


In [ ]:
row = generated_ids[0, input_ids['input_ids'].shape[1]:]
y_ = redistribute_codes(row)

In [ ]:
import IPython.display as ipd
ipd.Audio(y_, rate = 24000)